# <a target="_blank" href="https://miptstats.github.io/courses/ad_mipt.html">Phystech@DataScience</a>


In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.3)

### Линейная регрессия: практика
Перед началом работы догрузим необходимые нам функции



In [4]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

Рассмотрим данные исследования 2004 года о зависимости риска развития диабета от пола, возраста, индекса массы тела, среднего кровяного давления и других показателей. 



#### Загрузка данных

In [9]:
data = load_diabetes()
#data.head()

Функция `sklearn.datasets.load_diabetes()` возвращает словарь. В поле `data` записана матрица регрессоров, в которой данные предварительно центрированы и нормированы. В поле `target` записана мера прогрессирования заболевания в течении года. В поле `DESCR` можно прочитать подробнее о данных.

Посмотрим на описание датасета.

In [6]:
print(data['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

Поле `data` содержит матрицу размера 442 $\times$ 10, где 442 — количество пациентов, а 10 — количество признаков (возраст, пол, и т.д.). 
Строки матрицы соответствуют пациентам, столбцы — признакам.

In [7]:
data['data'].shape

(442, 10)

Целевая переменная $-$ мера прогрессирования заболевания в течении года.

In [8]:
data['target'].shape

(442,)

Создайте матрицу регрессоров $X$ (data) и столбец наблюдений $y$ (целевая переменная).

In [14]:
X, y = data['data'], data['target']

#### Обучение моделей

Разбейте данные случайно на две части — обучающую и тестовую в соотношении 80:20.

Если что-то забыли или что-то не понятно, можно почитать справку: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

При разбиении датасета стоит зафиксировать случайность для воспроизводимости результатов, поставив `random_state=42` в функцию разбиения (можете ставить любое число, необязательно именно 42)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(353, 10) (89, 10) (353,) (89,)


Заведите модель линейной регрессии из `sklearn` и обучите ее по обучающей части данных.

In [19]:
model = LinearRegression(fit_intercept=True)  # объявляем модель
model.fit(X_train, y_train)  # обучаем

LinearRegression()

Посмотрите на результат обучения. Выведите коэффициенты перед признаками и свободный коэффициент.

In [26]:
model.coef_, model.intercept_

(array([  37.90031426, -241.96624835,  542.42575342,  347.70830529,
        -931.46126093,  518.04405547,  163.40353476,  275.31003837,
         736.18909839,   48.67112488]),
 151.3456553477407)

Выполните предсказание модели на тестовой выборке.

In [28]:
y_pred = model.predict(X_test)

Реализуйте метрики MSE, MAE, MAPE без использования `sklearn` и других готовых реализаций.

Пусть $Y_1, ..., Y_n$ &mdash; истинные значения, а $\widehat{Y}_1, ..., \widehat{Y}_n$ &mdash; предсказания.


Метрика MSE (mean squared error) определяется как
$$MSE = \frac{1}{n}\sum_{i=1}^n \left(Y_i - \widehat{Y}_i\right)^2.$$

Метрика **MAE** (*mean absolute error*), определяемая как 
$$MAE = \frac{1}{n}\sum_{i=1}^n \left|Y_i - \widehat{Y}_i\right|.$$

Метрика **MAPE** (*mean absolute percentage error*), определяемая как 
$$MAPE = 100\% \cdot \frac{1}{n}\sum_{i=1}^n \frac{\left|Y_i - \widehat{Y}_i\right|}{Y_i}.$$


In [29]:
def mean_squared_error(y_true, y_pred):
    return 1/len(y_true)*sum((y_true - y_pred)**2)

def mean_absolute_error(y_true, y_pred):
    return 1/len(y_true)*sum(abs(y_true - y_pred))

def mean_absolute_percentage_error(y_true, y_pred):
    return 1/len(y_true)*sum(abs(y_true - y_pred)/y_true) * 100

Посчитайте MSE, MAE, MAPE на тестовой выборке и выведите с точностью до трех знаков после запятой.

In [35]:
round(mean_squared_error(y_test, y_pred), 3), round(mean_absolute_error(y_test, y_pred), 3), round(mean_absolute_percentage_error(y_test, y_pred), 3)

(2900.173, 42.794, 37.5)

посмотрим на точность на тренировочной выборке

In [40]:
y_pred_train = model.predict(X_train)
mean_absolute_percentage_error(y_train, y_pred_train)

38.69968073567731

### Различные разбиения

Разбейте данные на тренировочную и тестовую выборки еще раз: измените пераметр `random_state`. Для каждого из разбиений обучите модель и получите метрики качества. Меняются ли эти метрики, если менять разбиение данных?

In [38]:
mse = []
mae = []
mape = []
for state in [1, 495, 324, 8, 0, 900]:

    # разбиение данных
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=state)
    
    # обучение модели
    model = LinearRegression(fit_intercept=True)  # объявляем модель
    model.fit(X_train, y_train)  # обучаем

    # предсказание модели
    y_pred = model.predict(X_test)

    # подсчет метрики на тестовой выборке
    mse.append(mean_squared_error(y_test, y_pred))
    mae.append(mean_absolute_error(y_test, y_pred))
    mape.append(mean_absolute_percentage_error(y_test, y_pred))

In [39]:
mse, mae, mape

([2992.5576814529445,
  2794.9067662986104,
  3010.6302505696985,
  2937.84444243104,
  3424.316688213733,
  2471.145371497386],
 [41.97487568546231,
  41.751399130760994,
  45.0985139352397,
  44.44183760893984,
  46.1742024187542,
  41.13248524045231],
 [32.494165035046976,
  38.73452430268499,
  41.31397165904247,
  38.98915659717891,
  38.045957443894075,
  40.647877417347665])

**Вывод:**

обучили модель на тренировочной выборке, она научилась предсказывать целевой параметр, что проверили на тестовой выборке (ошибка около 37.5%), тренировочную выборку предсказывает с похожей ошибкой 38.7% (out[40]), таким образом не наблюдаем переобучения.
при разбиении данных есть параметр random_state который однозначно задает разбиение, соответственно в эксперименте выше где мы меняли, каждый раз обучали новые модели у которых слегка варьировались метрики, таким образом модель зависит от тренировочной выборки.
почему часто 42?